# Prepare EEG data for training of machine-learning models
+ Import data.
+ Apply filters (bandpass).
+ Detect potential bad channels and replace them by interpolation.
+ Detect potential bad epochs and remove them.

## Import packages & links

In [1]:
# Import packages
import os
import sys
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
sys.path.insert(0, os.path.dirname(os.getcwd()))

import mne
#%matplotlib inline
#from mayavi import mlab

In [2]:
from config import ROOT, PATH_CODE, PATH_DATA, PATH_OUTPUT, PATH_METADATA
PATH_CNTS = os.path.join(PATH_DATA, "17mnd mmn")

In [3]:
filename_labels = PATH_METADATA + "Screening_children5a_summary_new.txt" 
metadata = pd.read_csv(filename_labels, sep='\t')
metadata.head()

,id_child,groupDDP,atRiskOrNotDDP,dyslexicAtMidGroup3DDP,assignment1,assignment2,assignment3,assignment4,childInfoPresent,relativeInfoPresent,mmr_2mth,mmr_5mth,mmr_11mth,mmr_17mth,mmr_23mth,mmr_29mth,mmr_35mth,mmr_41mth,mmr_47mth
0,001,4,unclear,1,notEnoughInfo,notEnoughInfo,notEnoughInfo,notEnoughInfo,1,1,1,0,1,1,1,1,1,1,1
1,002,missing,missing,missing,notAtRisk_highestScores,notAtRisk_highestScores,notAtRisk_highestScores,notAtRisk_highestScores,0,1,0,0,0,0,1,1,1,0,0
2,003,3Ctrl,notAtRisk,0,notAtRisk_highestScores,notAtRisk_highestScores,notAtRisk_highestScores,notAtRisk_highestScores,1,1,1,0,1,0,1,0,1,1,0
3,004,missing,missing,missing,notAtRisk_highestScores,notAtRisk_highestScores,notAtRisk_highestScores,notAtRisk_highestScores,1,1,1,1,1,0,1,1,0,1,0
4,005,3Ctrl,notAtRisk,0,notAtRisk_highestScores,notAtRisk_highestScores,notAtRisk_highestScores,notAtRisk_highestScores,1,1,1,0,1,1,0,1,1,1,0


In [4]:
metadata.shape

(336, 19)

## Search all *.cnt files and check for how many we have a label

In [5]:
import fnmatch
import warnings
warnings.filterwarnings('ignore')

import helper_functions

dirs = os.listdir(PATH_CNTS)
cnt_files = fnmatch.filter(dirs, "*.cnt")

In [6]:
found_ids = [x[:3] for x in cnt_files]
idx = np.where(np.array(found_ids) == '036')[0]
[cnt_files[x] for x in idx]

['036_17_mc_mmn25_wk.cnt', '036_17_mc_mmn36.cnt']

In [7]:
metadata[metadata['id_child'] == '036']['groupDDP'].values[0]

'3Ctrl'

In [15]:
labels = []
np.random.seed(0)

found_ids = [x[:3] for x in cnt_files]
for ID in list(set(found_ids)): 
    idx = np.where(np.array(found_ids) == ID)[0]
    filenames = [cnt_files[x] for x in idx]
    label = metadata[metadata['id_child'] == ID]['groupDDP'].values[0]
    label_risk = metadata[metadata['id_child'] == ID]['assignment4'].values[0]
    if label == '1FRdys':
        label = 1
    elif label == '2FRndys':
        label = 0
    elif label == '3Ctrl': #TODO: check if this is correct!
        label = 0
    labels.append([ID, label, label_risk, filenames])

In [16]:
labels[:10]

[['156',
  1,
  'notAtRisk_rest',
  ['156_17_jr_mmn25_wk.cnt',
   '156_17_jr_mmn36_wk.cnt',
   '156_17_jr_mmn47_mixed.cnt',
   '156_17_jr_mmn58_slp.cnt']],
 ['750', 0, 'atRisk', ['750-462-17m-mr-mmn36.cnt']],
 ['613',
  0,
  'notAtRisk_rest',
  ['613-176-17m-mc-mmn25.cnt', '613-176-17m-mc-mmn36.cnt']],
 ['140',
  1,
  'notEnoughInfo',
  ['140_17_mr_mmn25_wk.cnt',
   '140_17_mr_mmn36_2.cnt',
   '140_17_mr_mmn36_wk.cnt',
   '140_17_mr_mmn47.cnt']],
 ['472', 1, 'notEnoughInfo', ['472_17_jd_mmn36_wk.cnt']],
 ['336', 0, 'notAtRisk_rest', ['336_17_mc_mmn36_wk.cnt']],
 ['616',
  0,
  'notAtRisk_highestScores',
  ['616-240-17m-mc-mmn25.cnt', '616-240-17m-mc-mmn36.cnt']],
 ['624',
  '4',
  'notAtRisk_rest',
  ['624-373-17m-mc-mmn25-waak.cnt', '624-373-17m-mc-mmn36-waak.cnt']],
 ['039', 0, 'notAtRisk_highestScores', ['039_17_jc_mmn36.cnt']],
 ['488',
  0,
  'notEnoughInfo',
  ['488_17_jd_mmn25_wk.cnt', '488_17_jd_mmn36_wk.cnt']]]

In [10]:
len(labels), len(list(set(found_ids)))

(247, 247)

### Count number (and type) of labels found:

In [11]:
labels_known = 0
labels_unknown = 0
labels_type = []

for x in labels:
    if x[1] == 1: #'dyslexic'
        labels_known += 1
        labels_type.append(1)
    elif x[1] == 0: #'non-dyslexic'
        labels_known += 1
        labels_type.append(0)
    else: # missing or unclear
        labels_unknown += 1  
        labels_type.append('missing')
        
print("Data with proper labels:", labels_known, "||| Data without proper label:", labels_unknown)     

Data with proper labels: 209 ||| Data without proper label: 38


In [12]:
print("Data for 'dyslexic':", labels_type.count(1))
print("Data for 'non-dyslexic':", labels_type.count(0))

Data for 'dyslexic': 49
Data for 'non-dyslexic': 160


In [13]:
# Check types of risk group labels found
labels_risktype = [x[2] for x in labels]
list(set(labels_risktype))

['missing',
 'atRisk',
 'notAtRisk_highestScores',
 'notEnoughInfo',
 'notAtRisk_rest']

In [14]:
metadata['atRiskOrNotDDP'][:10]

0      unclear
1      missing
2    notAtRisk
3      missing
4    notAtRisk
5    notAtRisk
6    notAtRisk
7    notAtRisk
8    notAtRisk
9      unclear
Name: atRiskOrNotDDP, dtype: object

In [17]:
group_notrisk = np.array(1*((metadata['atRiskOrNotDDP'] == 'notAtRisk')
                   | (metadata['assignment4'].isin(['notAtRisk_rest', 'notAtRisk_highestScores']))))

group_risk = np.array(1*((metadata['atRiskOrNotDDP'] == 'atRisk')
                   | (metadata['assignment4'] == 'at risk')))

In [18]:
np.sum(group_risk) + np.sum(group_notrisk)

307

In [19]:
label_risk = group_notrisk + 2*group_risk 
label_risk[label_risk == 3] = 2
label_risk = label_risk -1

In [20]:
label_risk

array([-1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0, -1,  0,  0,  0,
        0,  0, -1, -1,  1, -1,  1,  0, -1,  0,  1,  1,  1, -1, -1,  0,  1,
        1,  0,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1, -1, -1,  1, -1,
        1,  1,  0,  1,  1,  1,  1,  1,  0, -1,  1,  1,  1,  1,  1, -1,  1,
        1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1, -1,  1,  1,  1,  1,
        1,  1, -1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  0,  0,  0,  0,
        0, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0, -1,
        0,  0,  0,  0,  0,  0, -1, -1,  1,  1, -1,  0, -1,  1,  1,  1,  1,
        1,  1,  0,  1,  1,  1,  1, -1, -1, -1,  1,  1,  1,  0,  1,  1,  1,
        1,  1,  1,  1, -1,  1,  1,  0, -1, -1,  1,  1,  1,  0,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,
        1,  1,  1,  1,  1

In [21]:
group_notdys = np.array(1*(metadata['groupDDP'].isin(['1FRdys', '3Ctrl'])))

group_dys = np.array(1*(metadata['groupDDP'] == '2FRndys'))

In [22]:
np.sum(group_notdys) + np.sum(group_dys)

252

In [23]:
label_dys = group_notdys + 2*group_dys 
label_dys[label_dys == 3] = 2
label_dys = label_dys -1

In [24]:
label_dys

array([-1, -1,  0, -1,  0,  0,  0,  0,  0, -1, -1, -1, -1,  0,  0,  0,  0,
        0,  0,  0, -1,  0,  0,  0,  0, -1,  0,  0,  0, -1, -1,  0,  0,  0,
        0,  0, -1, -1,  1, -1,  1, -1, -1, -1,  1,  1,  0, -1, -1, -1,  0,
        0, -1,  1,  1,  0,  0,  1,  1, -1,  0,  0,  1,  1, -1, -1,  1, -1,
        0,  1, -1,  1,  1,  0,  1,  0, -1, -1,  0,  1,  1,  0,  1, -1,  1,
        1,  1,  0,  1,  0,  1,  0,  1, -1,  1,  0,  1, -1,  0,  1,  0,  0,
        1,  0, -1,  0,  0, -1,  0,  1,  1,  1,  1,  1,  1,  0,  0, -1, -1,
        0, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,
        0,  0,  0, -1,  0,  0,  0, -1, -1, -1,  0,  0,  0,  0,  0,  0, -1,
       -1,  0,  0,  0,  0,  0, -1, -1,  1,  0, -1,  0, -1,  0,  0,  0,  0,
        0,  0, -1,  1,  1,  1,  1, -1, -1, -1,  1,  0,  1, -1,  1,  0,  0,
        1,  1,  0,  1, -1,  1,  0, -1, -1, -1,  1,  0,  1,  0,  1,  0,  1,
        1,  1,  1,  0,  0,  1,  0,  1,  1,  1,  0,  1, -1,  1,  1,  1,  0,
        1,  1,  0,  1,  1

## create Dataframe with labels to be used

In [25]:
labels_final = pd.DataFrame(data=metadata['id_child'].values, columns=['id_child'])
labels_final['label_dys'] = label_dys
labels_final['label_risk'] = label_risk
labels_final.head()

,id_child,label_dys,label_risk
0,001,-1,-1
1,002,-1,0
2,003,0,0
3,004,-1,0
4,005,0,0


In [26]:
print("Data for 'at risk':", labels_risktype.count('atRisk'))
print("Data for 'notAtRisk_rest':", labels_risktype.count('notAtRisk_rest'))
print("Data for 'notAtRisk_highestScores':", labels_risktype.count('notAtRisk_highestScores'))

Data for 'at risk': 91
Data for 'notAtRisk_rest': 56
Data for 'notAtRisk_highestScores': 49


In [27]:
labels_risktype = [x[2] for x in labels if x[1] in [1,0]]
print("Data for 'at risk':", labels_risktype.count('atRisk'))
print("Data for 'notAtRisk_rest':", labels_risktype.count('notAtRisk_rest'))
print("Data for 'notAtRisk_highestScores':", labels_risktype.count('notAtRisk_highestScores'))

Data for 'at risk': 79
Data for 'notAtRisk_rest': 43
Data for 'notAtRisk_highestScores': 45


In [28]:
metadata.loc[(metadata['groupDDP'].isin(['1FRdys', '2FRndys', '3Ctrl']) 
              & metadata['assignment4'].isin(['at risk', 'notAtRisk_rest', 'notAtRisk_highestScores', ]))]

,id_child,groupDDP,atRiskOrNotDDP,dyslexicAtMidGroup3DDP,assignment1,assignment2,assignment3,assignment4,childInfoPresent,relativeInfoPresent,mmr_2mth,mmr_5mth,mmr_11mth,mmr_17mth,mmr_23mth,mmr_29mth,mmr_35mth,mmr_41mth,mmr_47mth
2,003,3Ctrl,notAtRisk,0,notAtRisk_highestScores,notAtRisk_highestScores,notAtRisk_highestScores,notAtRisk_highestScores,1,1,1,0,1,0,1,0,1,1,0
4,005,3Ctrl,notAtRisk,0,notAtRisk_highestScores,notAtRisk_highestScores,notAtRisk_highestScores,notAtRisk_highestScores,1,1,1,0,1,1,0,1,1,1,0
5,006,3Ctrl,notAtRisk,0,notAtRisk_highestScores,notAtRisk_highestScores,notAtRisk_highestScores,notAtRisk_highestScores,1,1,1,0,0,1,1,1,0,1,0
8,009,3Ctrl,notAtRisk,1,notEnoughInfo,notEnoughInfo,notEnoughInfo,notAtRisk_rest,1,1,1,0,1,1,1,1,1,1,0
15,016,3Ctrl,notAtRisk,0,notAtRisk_highestScores,notAtRisk_highestScores,notAtRisk_highestScores,notAtRisk_highestScores,1,1,1,1,1,0,0,0,0,0,0
16,017,3Ctrl,notAtRisk,0,notAtRisk_highestScores,notAtRisk_highestScores,notAtRisk_highestScores,notAtRisk_highestScores,1,1,1,0,1,1,1,0,0,1,0
17,018,3Ctrl,notAtRisk,0,notAtRisk_highestScores,notAtRisk_highestScores,notAtRisk_highestScores,notAtRisk_highestScores,1,1,1,0,1,1,1,1,1,1,0
19,021,3Ctrl,notAtRisk,0,notAtRisk_highestScores,notAtRisk_highestScores,notAtRisk_highestScores,notAtRisk_highestScores,1,1,1,0,1,1,1,1,1,1,0
21,023,3Ctrl,notAtRisk,0,notAtRisk_highestScores,notAtRisk_highestScores,notAtRisk_highestScores,notAtRisk_highestScores,1,1,1,0,1,0,1,1,1,1,0
22,024,3Ctrl,notAtRisk,0,notAtRisk_rest,notAtRisk_rest,notAtRisk_rest,notAtRisk_rest,1,1,1,0,1,0,1,1,0,1,0


# Workflow data processing
1. Load cnt files.
2. Select same number of channels (here: 30 same channels which exist for both 30 and 62 channel data)
3. Preprocess raw data (bandpass + detect outliers and 'bad' epochs).
4. Store epoch data and event type as array

## LABELS:
+ After Karin's search we have proper labels for much more files!  


In [29]:
from importing import read_cnt_file, standardize_EEG

In [38]:
# Initialize array
signal_collection = np.zeros((0,30,501)) #62
label_collection = []
ID_collection = []
metadata_collection = []

collect_in_one_array = False

for i, filename in enumerate(cnt_files):
    
    # First check if we have proper label for that file
    # -----------------------------------------------------------
    
    ID = filename[:3]
    label = labels_final[labels_final['id_child'] == ID]['label_dys'].values[0]
    label_risk = labels_final[labels_final['id_child'] == ID]['label_risk'].values[0]
    #label = metadata[metadata['id_child'] == ID]['groupDDP'].values[0]
    #label_risk = metadata[metadata['id_child'] == ID]['assignment4'].values[0]
    
    if (label < 0) or (label_risk < 0):
        print("No proper label found for file: ", filename)
    else:
        #label_group = int(metadata[metadata["file"].str.match(filename[:-4])]['group'])
        label_group = 'dys' + str(label) + '_risk' + str(label_risk)
        
        print(40*"=")
        print("Importing file: ",filename)
        print("Data belongs into group: ", label_group)

        # Import data and events
        file = os.path.join(PATH_CNTS, filename)

        signal_collect, label_collect, ch_names = read_cnt_file(file, 
                                                                  label_group,
                                                                  event_idx = [3, 13, 66],
                                                                  channel_set = "30",
                                                                  tmin = -0.2,
                                                                  tmax = 0.8,
                                                                  lpass = 0.5, 
                                                                  hpass = 40, 
                                                                  threshold = 5, 
                                                                  max_bad_fraction = 0.2)
        
        
        # Standardize data
        # --------------------------------------------------------
        if signal_collect is not None:
            signal_collect = standardize_EEG(signal_collect,
                                 std_aim = 1,                   
                                 centering = 'per_channel',
                                 scaling = 'global')
        
        # Save data and labels
        # ---------------------------------------------------------
        if signal_collect is not None:
            
            if collect_in_one_array:

                # Get signals as array and add to total collection
                print(signal_collect.shape, len(label_collect))
                signal_collection = np.concatenate((signal_collection, signal_collect), axis=0)
                label_collection += label_collect

            else:
                if len(label_collect) > 1:
                #if label_collect is not None:
                    file = os.path.join(PATH_OUTPUT, "processed_data_" + filename[:-4] + ".npy")
                    np.save(file, signal_collect)

                    #filename = os.path.join(PATH_OUTPUT, "EEG_data_30ch_1s_corrected_metadata_ID"+ ID + ".csv")
                    file = os.path.join(PATH_OUTPUT, "processed_data_" + filename[:-4] + ".csv")

                    with open(file, 'w', newline='') as csvFile:
                        writer = csv.writer(csvFile)
                        writer.writerow(label_collect)
                    csvFile.close()
            
            ID_collection += [ID] * len(label_collect) 
            metadata_collection.append((i, filename, signal_collection.shape[0], ch_names))

No proper label found for file:  001_17_jc_mmn36_slp_mmn25_slp_mmn47_slp_mmn58_slp.cnt
Importing file:  005_17_jc_mmn36_slp_mmn25_slp_mmn47_mixed.cnt
Data belongs into group:  dys0_risk0
Reading 0 ... 1120199  =      0.000 ...  2240.398 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (6.602 sec)

Used Annotations descriptions: ['0', '12', '13', '14', '2', '3', '4', '55', '66', '77']
No outliers found with given threshold.
No outliers found with given threshold.
No outliers found wit

- Filter length: 3301 samples (6.602 sec)

Used Annotations descriptions: ['13', '3', '66']
Found 6 bad epochs in a total of 6  channels.
Marked 6 bad epochs in a total of 400  epochs.
Found 2 bad epochs in a total of 2  channels.
Marked 2 bad epochs in a total of 50  epochs.
No outliers found with given threshold.
Importing file:  017_17_jc_mmn25_wk.cnt
Data belongs into group:  dys0_risk0
Reading 0 ... 369359  =      0.000 ...   738.718 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samp

- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (6.602 sec)

Used Annotations descriptions: ['0', '13', '3', '66']
Found 400 bad epochs in a total of 25  channels.
Found bad channel (more than 80.0  bad epochs): Channel no:  2
Found bad channel (more than 80.0  bad epochs): Channel no:  6
Found bad channel (more than 80.0  bad epochs): Channel no:  8
Marked 123 bad epochs in a total of 400  epochs.
----------------------------------------
Found too many bad channels (3)
Importing file:  027_17_jc_mmn25.cnt
Data belongs into group:  dys0_risk0
Reading 0 ... 379699  =      0.000 ...   759.398 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed 

- Filter length: 3301 samples (6.602 sec)

Used Annotations descriptions: ['1', '12', '13', '33']
Found 3 bad epochs in a total of 5  channels.
Marked 3 bad epochs in a total of 50  epochs.
Importing file:  030_17_jc_mmn25_wk.cnt
Data belongs into group:  dys0_risk0
Reading 0 ... 378419  =      0.000 ...   756.838 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (6.602 sec)

Used Annotations descriptions: ['12', '2', '55']
Importing file:  030_17_jc_mmn36_wk.cnt
Data belongs into gro

Found 6 bad epochs in a total of 2  channels.
Marked 6 bad epochs in a total of 50  epochs.
Found 7 bad epochs in a total of 5  channels.
Marked 7 bad epochs in a total of 50  epochs.
Importing file:  034_17_mc_mmn36_slp.cnt
Data belongs into group:  dys0_risk0
Reading 0 ... 382579  =      0.000 ...   765.158 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (6.602 sec)

Used Annotations descriptions: ['0', '13', '3', '66']
Found 2 bad epochs in a total of 2  channels.
Marked 2 bad ep

...

- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (6.602 sec)

Used Annotations descriptions: ['0', '13', '3', '66']
Found 28 bad epochs in a total of 12  channels.
Marked 28 bad epochs in a total of 400  epochs.
Found 5 bad epochs in a total of 5  channels.
Marked 5 bad epochs in a total of 50  epochs.
Found 3 bad epochs in a total of 6  channels.
Marked 3 bad epochs in a total of 50  epochs.
Importing file:  729-141-17m-jr-mmn36_2.cnt
Data belongs into group:  dys1_risk1
Reading 0 ... 367859  =      0.000 ...   735.718 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.2

Found 24 bad epochs in a total of 12  channels.
Marked 24 bad epochs in a total of 480  epochs.
Found 2 bad epochs in a total of 2  channels.
Marked 2 bad epochs in a total of 60  epochs.
No outliers found with given threshold.
Importing file:  735-125-17m-mr-mmn25.cnt
Data belongs into group:  dys1_risk1
Reading 0 ... 458639  =      0.000 ...   917.278 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (6.602 sec)

Used Annotations descriptions: ['0', '12', '2', '55']
Importing file: 

- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (6.602 sec)

Used Annotations descriptions: ['0', '13', '3', '66']
Found 28 bad epochs in a total of 24  channels.
Marked 28 bad epochs in a total of 800  epochs.
Found 5 bad epochs in a total of 6  channels.
Marked 5 bad epochs in a total of 100  epochs.
Found 3 bad epochs in a total of 3  channels.
Marked 3 bad epochs in a total of 100  epochs.
No proper label found for file:  740-396-17m-jr-mmn25-waak.cnt
No proper label found for file:  740-396-17m-jr-mmn36-waak.cnt
Importing file:  741-405-17m-mr-mmn36.cnt
Data belongs into group:  dys0_risk1
Reading 0 ... 1202519  =      0.000 ...  2405.038 secs...
Filtering raw data in 1 contig

- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (6.602 sec)

Used Annotations descriptions: ['0', '13', '3', '66']
Found 111 bad epochs in a total of 15  channels.
Marked 111 bad epochs in a total of 800  epochs.
Found 10 bad epochs in a total of 6  channels.
Marked 10 bad epochs in a total of 100  epochs.
Found 14 bad epochs in a total of 7  channels.
Marked 14 bad epochs in a total of 100  epochs.
No proper label found for file:  751-542-17m-jr-mmn36.cnt
Importing file:  752-457-17m-jr-mmn36.cnt
Data belongs into group:  dys0_risk1
Reading 0 ... 741059  =      0.000 ...  1482.118 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-doma

Found 10 bad epochs in a total of 3  channels.
Marked 10 bad epochs in a total of 50  epochs.
Importing file:  757-487-17m-jr-mmn36.cnt
Data belongs into group:  dys1_risk1
Reading 0 ... 743799  =      0.000 ...  1487.598 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (6.602 sec)

Used Annotations descriptions: ['0', '13', '3', '66']
Found 99 bad epochs in a total of 14  channels.
Marked 99 bad epochs in a total of 800  epochs.
Found 13 bad epochs in a total of 7  channels.
Marked 

In [52]:
label_collect[:10]

['3dys1_risk1',
 '3dys1_risk1',
 '3dys1_risk1',
 '3dys1_risk1',
 '3dys1_risk1',
 '3dys1_risk1',
 '3dys1_risk1',
 '3dys1_risk1',
 '3dys1_risk1',
 '3dys1_risk1']

# Save entire processed dataset:
--> for this: run above code with ``collect_in_one_array = True``

In [75]:
filename = os.path.join(PATH_OUTPUT, "EEG_data_30channels_1s_corrected.npy")
np.save(filename, signal_collection)

filename = os.path.join(PATH_OUTPUT, "EEG_data_30channels_1s_corrected_labels.npy")
np.save(filename, label_collection)

import csv
filename = os.path.join(PATH_OUTPUT, "EEG_data_30channels_1s_corrected_metadata.csv")

with open(filename, 'w') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerows(metadata_collection)
csvFile.close()